In [1]:
import tushare as ts
from typing import Any

import numpy as np
import pandas as pd

pro = ts.pro_api('2026c96ef5fa7fc3241c96baafd638c585284c7fefaa00b93ef0a62c')

def code_to_secid(df : pd.DataFrame , code_col = 'ts_code' , retain = False):
    '''switch old symbol into secid'''
    if code_col not in df.columns.values: return df
    replace_dict = {'T00018' : '600018'}
    df['secid'] = df[code_col].astype(str).str.slice(0, 6).replace(replace_dict)
    df['secid'] = df['secid'].where(df['secid'].str.isdigit() , '-1').astype(int)
    if not retain: del df[code_col]
    return df

In [1]:
from src.data.fetcher.tushare.task import main
main()

Calendar Already Updated at 20240702
Description Already Updated at 20240702
SWIndustry Already Updated at 20240702
THSConcept Already Updated at 20240702


In [11]:
import tushare as ts
from typing import Any

import numpy as np
import pandas as pd

pro = ts.pro_api('2026c96ef5fa7fc3241c96baafd638c585284c7fefaa00b93ef0a62c')

def code_to_secid(df : pd.DataFrame , code_col = 'ts_code' , retain = False):
    '''switch old symbol into secid'''
    if code_col not in df.columns.values: return df
    replace_dict = {'T00018' : '600018'}
    df['secid'] = df[code_col].astype(str).str.slice(0, 6).replace(replace_dict)
    df['secid'] = df['secid'].where(df['secid'].str.isdigit() , '-1').astype(int)
    if not retain: del df[code_col]
    return df

df = code_to_secid(df)
df['start_date'] = df['start_date'].fillna(-1).astype(int)
df['ann_date'] = df['ann_date'].fillna(-1).astype(int)
df['end_date'] = df['end_date'].fillna(99991231).astype(int)
df['st'] = np.where(df['ann_date'] > 0 , np.minimum(df['start_date'] , df['ann_date']) , df['ann_date'])
df['ed'] = df['end_date']

In [9]:
import baostock as bsc
dir(bsc)

['__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'common',
 'cons',
 'corpreport',
 'data',
 'evaluation',
 'login',
 'logout',
 'macroscopic',
 'metadata',
 'query_adjust_factor',
 'query_all_stock',
 'query_balance_data',
 'query_cash_flow_data',
 'query_deposit_rate_data',
 'query_dividend_data',
 'query_dupont_data',
 'query_forecast_report',
 'query_growth_data',
 'query_history_k_data_plus',
 'query_hs300_stocks',
 'query_loan_rate_data',
 'query_money_supply_data_month',
 'query_money_supply_data_year',
 'query_operation_data',
 'query_performance_express_report',
 'query_profit_data',
 'query_required_reserve_ratio_data',
 'query_stock_basic',
 'query_stock_industry',
 'query_sz50_stocks',
 'query_trade_dates',
 'query_zz500_stocks',
 'security',
 'util']

In [2]:
import baostock as bsc
import datetime
import pandas as pd
 
#开始日期，默认为1999-11-11，baostock平台只有该日期后的数据
start_date='2024-01-01'
#截止日期，默认为今日
end_date=datetime.date.today().strftime('%Y-%m-%d')
#股票代码
stock_code='sh.600000'
#获取数据频率
frequency='5'
#复权标志
adjustflag='3'
 
def get_stock_5min_data_c(bsa,stock_number,start_day,end_day,freq,adflag):
 
    print('==========================================================')
 
    #####get stock data#####
    rs=bsa.query_history_k_data_plus(stock_number, "date,time,open,high,low,close,volume,amount,adjustflag",start_date=start_day,end_date=end_day,frequency=freq, adjustflag=adflag)
 
    print('请求历史数据返回代码:'+rs.error_code)
    print('请求历史数据返回信息:'+rs.error_msg)
 
    data_list = rs.get_data()
    result=pd.DataFrame(data_list,columns=rs.fields)
    return result

bsc.login()
 
stock_trade_result = get_stock_5min_data_c(bsc, stock_code , start_date, end_date, frequency,adjustflag)
 
bsc.logout()

login success!
请求历史数据返回代码:0
请求历史数据返回信息:success
logout success!


In [9]:
stock_code

'sh.600000'

In [3]:
stock_trade_result

,date,time,open,high,low,close,volume,amount,adjustflag
0,2024-01-02,20240102093500000,6.6300,6.6400,6.6100,6.6200,1902300,12603192.0000,3
1,2024-01-02,20240102094000000,6.6100,6.6300,6.6100,6.6300,692800,4584196.0000,3
2,2024-01-02,20240102094500000,6.6200,6.6300,6.6100,6.6200,613200,4059260.0000,3
3,2024-01-02,20240102095000000,6.6200,6.6400,6.6200,6.6400,1124800,7457586.0000,3
4,2024-01-02,20240102095500000,6.6400,6.6500,6.6300,6.6300,690900,4587764.0000,3
...,...,...,...,...,...,...,...,...,...
5755,2024-07-03,20240703144000000,8.5300,8.5400,8.5200,8.5400,825400,7042848.0000,3
5756,2024-07-03,20240703144500000,8.5300,8.5400,8.5300,8.5400,341700,2916384.0000,3
5757,2024-07-03,20240703145000000,8.5400,8.5500,8.5300,8.5500,761200,6501568.0000,3
5758,2024-07-03,20240703145500000,8.5500,8.5500,8.5400,8.5400,721300,6164352.0000,3


In [15]:
stock_trade_result

""


In [22]:
import baostock as bs
import pandas as pd

start_date='2024-01-01'
#截止日期，默认为今日
end_date=datetime.date.today().strftime('%Y-%m-%d')

bs.login()

# 获取指定日期的指数、股票数据
stock_rs = bs.query_all_stock(date)
stock_df = stock_rs.get_data()
codes = stock_df["code"][:100]
data_df = []
for code in codes:
    k_rs = bs.query_history_k_data_plus(code, "date,code,open,high,low,close", date, date)
    data_df.append(k_rs.get_data())
bs.logout()



login success!
logout success!


In [10]:
df[df['change_reason'].isin(['终止上市', 'ST', '*ST', '暂停上市'])]

,name,start_date,end_date,ann_date,change_reason,secid
0,左江退,20240708,99991231,20240628,终止上市,300799
4,越博退,20240701,99991231,20240621,终止上市,300742
7,三盛退,20240627,99991231,20240620,终止上市,300282
33,太安退,20240614,99991231,20240606,终止上市,2433
51,中期退,20240606,99991231,20240531,终止上市,996
...,...,...,...,...,...,...
17452,ST湘中意,19980429,19990530,-1,ST,548
17453,ST黄河科,19980429,20020328,-1,ST,600831
17454,ST粤海发,19980429,20010610,-1,ST,600647
17455,ST辽房天,19980429,19990525,-1,ST,558
